<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/sample/elasticsearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install elasticsearch pandas transformers torch sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.6/524.6 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 4.0 MB/s eta 0:00:00


In [2]:
from elasticsearch import Elasticsearch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer

In [6]:
# Load the DataFrame
df = pd.read_excel('/content/drive/MyDrive/AIEngineer/elasticsearch/jobdb.xlsx')
df

,No,Position,Description,Responsibilities,Qualification
0,1,Network Engineer,Responsible for managing and maintaining the c...,"Design, implement, and maintain network system...","Bachelor’s degree in Computer Science, Informa..."
1,2,Software Developer,Develop and maintain web-based applications an...,"Write clean, scalable code for web application...","Bachelor’s degree in Computer Science, Softwar..."
2,3,IT Support Specialist,Provide technical assistance and support to co...,"Diagnose hardware, software, and network probl...",Diploma or Bachelor’s degree in Information Te...
3,4,Digital Marketing Specialist,"Plan, execute, and optimize digital marketing ...",Develop and manage online marketing campaigns ...,"Bachelor’s degree in Marketing, Communications..."
4,5,Project Manager,Oversee and coordinate the successful executio...,"Develop project plans, timelines, and budgets....","Bachelor’s degree in Business, IT, or a relate..."
5,6,System Administrator,Manage and maintain the company's server infra...,"Install, configure, and maintain server hardwa...","Bachelor’s degree in Information Technology, C..."
6,7,Business Analyst,Analyze business processes and data to provide...,Gather and document business requirements from...,"Bachelor’s degree in Business, Economics, or r..."
7,8,Cybersecurity Specialist,Ensure the company’s digital assets and networ...,Implement security measures and protocols.\nMo...,"Bachelor’s degree in Cybersecurity, Informatio..."
8,9,Data Scientist,Use advanced data analytics and machine learni...,"Collect, process, and analyze large datasets.\...","Bachelor’s or Master’s degree in Data Science,..."
9,10,Sales Manager,Lead the sales team and drive revenue growth b...,Develop and implement sales strategies.\nLead ...,"Bachelor’s degree in Business, Marketing, or r..."


In [9]:
df.columns

Index(['No', 'Position', 'Description', 'Responsibilities', 'Qualification'], dtype='object')

In [10]:
mappings = {
    "mappings": {
        "properties": {
            "No": {"type": "integer"},
            "Position": {"type": "text"},
            "Description": {"type": "text"},
            "Responsibilities": {"type": "text"},
            "Qualification": {"type": "text"},
            "embedding": {"type": "dense_vector","dims": 384}
            }
        }
    }

In [16]:
# Elasticsearch credentials
user = "natthaphol.po"
passwords = "cnEM5CeFrG"

# Elasticsearch setup with basic authentication and HTTPS
es = Elasticsearch(
    hosts=[{'host': 'elk.manageai.co.th', 'port': 443, 'scheme': 'https'}],
    basic_auth=("natthaphol.po", "cnEM5CeFrG")  # Ensure the credentials are correct
)

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [19]:
def test_elasticsearch_connection():
    try:
        es.ping()
        print("Connection successful!")
    except Exception as e:
        print(f"Error: {e}")

test_elasticsearch_connection()


Connection successful!


In [11]:
# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Function to encode text to embeddings
def rag_encode(text):
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
    # Convert tensor to flat list
    return sentence_embeddings.squeeze().tolist()  # Flatten to 1D list

## embed to es

In [23]:
# Create index with correct mapping
def create_index_if_not_exists(index_name):
    if not es.indices.exists(index=index_name):  # Only create if the index doesn't exist
        es.indices.create(index=index_name, body=mappings)

def index_row_by_row(df, index_name):
    create_index_if_not_exists(index_name)

    # Index each row individually
    for _, row in df.iterrows():
        try:
            # Prepare document for indexing
            document = {
                "No": row['No'],
                "Position": row['Position'],
                "Description": row['Description'],
                "Responsibilities": row['Responsibilities'],
                "Qualification": row['Qualification'],
                "embedding": rag_encode(f"{row['Position']} {row['Description']} {row['Responsibilities']} {row['Qualification']}")
            }

            # Index the document
            es.index(index=index_name, body=document)
        except Exception as e:
            print(f"Error indexing row: {row} - {str(e)}")

# Process and index the DataFrame row by row
index_row_by_row(df, 'internal-manageai-jobdb-index')

## Search

In [24]:
# Query Elasticsearch index
def query_elasticsearch(index_name, query_body):
    try:
        response = es.search(index=index_name, body=query_body)
        return response['hits']['hits']
    except Exception as e:
        print(f"Error querying index: {str(e)}")
        return []

# Example query to search documents by Position (job role)
def search_by_position(position, index_name):
    query_body = {
        "query": {
            "match": {
                "Position": position
            }
        }
    }

    results = query_elasticsearch(index_name, query_body)
    for result in results:
        print(f"Found document: {result['_source']}")


In [25]:
search_by_position("Software Developer", 'internal-manageai-jobdb-index')

Found document: {'No': 2, 'Position': 'Software Developer', 'Description': 'Develop and maintain web-based applications and services to support the company’s internal and external needs.', 'Responsibilities': 'Write clean, scalable code for web applications.\nCollaborate with product managers and designers to understand user requirements.\nDebug and troubleshoot software issues.\nDevelop APIs and integrate third-party services.\nMaintain documentation and support continuous integration/deployment.', 'Qualification': 'Bachelor’s degree in Computer Science, Software Engineering, or related field.\nProficient in programming languages such as Java, Python, or JavaScript.\nExperience with frameworks such as React, Node.js, or Django.\nStrong understanding of databases (SQL, NoSQL).\nAbility to work in an agile development environment.', 'embedding': [-0.020841365680098534, -0.006470450200140476, 0.015252257697284222, 0.04488835483789444, -0.022452576085925102, -0.08007542043924332, 0.012467

In [27]:
# Example query to search jobs by Position, Description, and Responsibilities
def search_job(position, description, responsibilities, index_name):
    query_body = {
        "query": {
            "bool": {
                "must": [
                    {"match": {"Position": position}},
                    {"match": {"Description": description}},
                    {"match": {"Responsibilities": responsibilities}}
                ]
            }
        }
    }

    results = query_elasticsearch(index_name, query_body)
    for result in results:
        print(f"Found document: {result['_source']}")


In [28]:
search_job("Software Developer", "web-based applications", "scalable code", 'internal-manageai-jobdb-index')

Found document: {'No': 2, 'Position': 'Software Developer', 'Description': 'Develop and maintain web-based applications and services to support the company’s internal and external needs.', 'Responsibilities': 'Write clean, scalable code for web applications.\nCollaborate with product managers and designers to understand user requirements.\nDebug and troubleshoot software issues.\nDevelop APIs and integrate third-party services.\nMaintain documentation and support continuous integration/deployment.', 'Qualification': 'Bachelor’s degree in Computer Science, Software Engineering, or related field.\nProficient in programming languages such as Java, Python, or JavaScript.\nExperience with frameworks such as React, Node.js, or Django.\nStrong understanding of databases (SQL, NoSQL).\nAbility to work in an agile development environment.', 'embedding': [-0.020841365680098534, -0.006470450200140476, 0.015252257697284222, 0.04488835483789444, -0.022452576085925102, -0.08007542043924332, 0.012467